In [ ]:
import time
import numpy as np
from scipy.sparse import random as sparse_random
from scipy.optimize import linprog

def generate_infeasible_problems(m=1000, n=1600, num_problems=10, density=1.0):
    problems = []
    for _ in range(num_problems):
        A_sparse = sparse_random(m, n, density=density, data_rvs=np.random.rand)
        A = np.abs(A_sparse.toarray())  # ensure A >= 0
        b = -np.random.rand(m)          # b <= 0
        c = np.ones(n)
        problems.append((A, b, c))
    return problems

def random_projector_normal(m, k):
    # k x m matrix with entries +1, 0, -1 with probabilities 1/6, 2/3, 1/6, scaled by 1/sqrt(k)
    return np.random.choice([1, 0, -1], size=(k, m), p=[1/6, 2/3, 1/6]) / np.sqrt(k)

# Example parameters
m, n, k = 1000, 1600, 333
densities = [0.1, 0.3, 0.5, 0.7]
num_problems = 10

results = []  # (m, n, dens, k, orgCPU, prjCPU, T_time, mult_time, solve_time, acc)

for dens in densities:
    problems = generate_infeasible_problems(m, n, num_problems, density=dens)
    org_times, prj_times = [], []
    T_times, mult_times, solve_times = [], [], []
    num_success = 0  # count where original infeasible but projected feasible
    total = len(problems)

    for (A, b, c) in problems:
        # Original solve timing
        t0 = time.perf_counter()
        res_orig = linprog(c, A_eq=A, b_eq=b, bounds=(0, None), method='highs')
        org_times.append(time.perf_counter() - t0)
        original_infeasible = (res_orig.status == 2)

        # Projected solve timing: separate timings for each step
        t1 = time.perf_counter()
        T = random_projector_normal(A.shape[0], k)
        t2 = time.perf_counter()
        A_proj = T @ A
        b_proj = T @ b
        t3 = time.perf_counter()
        res_proj = linprog(c, A_eq=A_proj, b_eq=b_proj, bounds=(0, None), method='highs')
        t4 = time.perf_counter()

        T_time = t2 - t1      # time for sampling T
        mult_time = t3 - t2   # time for matrix multiplications
        solve_time = t4 - t3  # time for solving LP
        prj_time = T_time + mult_time + solve_time

        T_times.append(T_time)
        mult_times.append(mult_time)
        solve_times.append(solve_time)
        prj_times.append(prj_time)

        projected_feasible = (res_proj.status == 0)
        if original_infeasible and projected_feasible:
            num_success += 1

    avg_org = np.mean(org_times)
    avg_prj = np.mean(prj_times)
    avg_T = np.mean(T_times)
    avg_mult = np.mean(mult_times)
    avg_solve = np.mean(solve_times)
    acc = num_success / total

    results.append((m, n, dens, k, avg_org, avg_prj, avg_T, avg_mult, avg_solve, acc))

# Print the results table
print(" m    n    dens    k    orgCPU   prjCPU   T_time   mult_time  solve_time  acc")
for (mm, nn, dens, kk, orgT, prjT, Tt, Mt, St, acc) in results:
    print(f"{mm:3d}  {nn:4d}  {dens:0.1f}  {kk:3d}   {orgT:.2f}   {prjT:.2f}   {Tt:.2f}   {Mt:.2f}   {St:.2f}   {acc:.2f}")



 m    n    dens    k    orgCPU   prjCPU   T_time   mult_time  solve_time  acc
1000  1600  0.1  333   0.06   1.01   0.01   0.06   0.94   0.00
1000  1600  0.3  333   0.08   0.81   0.01   0.04   0.76   0.00
1000  1600  0.5  333   0.10   0.72   0.01   0.03   0.68   0.00
1000  1600  0.7  333   0.16   0.37   0.01   0.05   0.31   0.00


In [ ]:
import time
import numpy as np
from scipy.sparse import random as sparse_random
from scipy.optimize import linprog

def generate_infeasible_problems(m=1000, n=1600, num_problems=10, density=1.0):
    problems = []
    for _ in range(num_problems):
        A_sparse = sparse_random(m, n, density=density, data_rvs=np.random.rand)
        A = np.abs(A_sparse.toarray())  # ensure A >= 0
        b = -np.random.rand(m)          # b <= 0
        c = np.ones(n)
        problems.append((A, b, c))
    return problems

def random_projector_orthogonal(m, k):
    # Generate an m x m orthogonal matrix via QR decomposition and take the first k rows.
    Q, _ = np.linalg.qr(np.random.randn(m, m))
    T = Q[:k, :]
    return T

# Example parameters
m, n, k = 1000, 1600, 333
densities = [0.1, 0.3, 0.5, 0.7]
num_problems = 10

results = []  # (m, n, dens, k, orgCPU, prjCPU, T_time, mult_time, solve_time, acc)

for dens in densities:
    problems = generate_infeasible_problems(m, n, num_problems, density=dens)
    org_times, prj_times = [], []
    T_times, mult_times, solve_times = [], [], []
    num_success = 0  # count where original infeasible but projected feasible
    total = len(problems)

    for (A, b, c) in problems:
        # Original solve timing
        t0 = time.perf_counter()
        res_orig = linprog(c, A_eq=A, b_eq=b, bounds=(0, None), method='highs')
        org_times.append(time.perf_counter() - t0)
        original_infeasible = (res_orig.status == 2)

        # Projected solve timing: separate timings for each step
        t1 = time.perf_counter()
        T = random_projector_orthogonal(A.shape[0], k)
        t2 = time.perf_counter()
        A_proj = T @ A
        b_proj = T @ b
        t3 = time.perf_counter()
        res_proj = linprog(c, A_eq=A_proj, b_eq=b_proj, bounds=(0, None), method='highs')
        t4 = time.perf_counter()

        T_time = t2 - t1      # time for sampling T
        mult_time = t3 - t2   # time for matrix multiplications
        solve_time = t4 - t3  # time for solving LP
        prj_time = T_time + mult_time + solve_time

        T_times.append(T_time)
        mult_times.append(mult_time)
        solve_times.append(solve_time)
        prj_times.append(prj_time)

        projected_feasible = (res_proj.status == 0)
        if original_infeasible and projected_feasible:
            num_success += 1

    avg_org = np.mean(org_times)
    avg_prj = np.mean(prj_times)
    avg_T = np.mean(T_times)
    avg_mult = np.mean(mult_times)
    avg_solve = np.mean(solve_times)
    acc = num_success / total

    results.append((m, n, dens, k, avg_org, avg_prj, avg_T, avg_mult, avg_solve, acc))

# Print the results table
print(" m    n    dens    k    orgCPU   prjCPU   T_time   mult_time  solve_time  acc")
for (mm, nn, dens, kk, orgT, prjT, Tt, Mt, St, acc) in results:
    print(f"{mm:3d}  {nn:4d}  {dens:0.1f}  {kk:3d}   {orgT:.2f}   {prjT:.2f}   {Tt:.2f}   {Mt:.2f}   {St:.2f}   {acc:.2f}")



 m    n    dens    k    orgCPU   prjCPU   T_time   mult_time  solve_time  acc
1000  1600  0.1  333   0.04   1.07   0.28   0.04   0.76   0.00
1000  1600  0.3  333   0.07   1.05   0.26   0.04   0.75   0.00
1000  1600  0.5  333   0.11   1.05   0.30   0.04   0.72   0.00
1000  1600  0.7  333   0.15   0.50   0.18   0.03   0.29   0.00
